# Imports

In [1]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean
from igraph import Graph as igraphGraph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance
from scipy.spatial.distance import cdist


# Input

In [2]:
transformation = Transformation()

user_number_triangles = 500   #à diminuer si le process est trop long
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [3]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [4]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [5]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [6]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[0.76419333 0.76463794 0.76419333 ... 0.71143825 1.0327106  0.71143825]
[0.72680424 0.7272271  0.72680424 ... 0.67663027 0.98218397 0.67663027]
[0.72656961 0.72668255 0.72656961 ... 0.71296363 0.78929482 0.71296363]
(5999,)


### Multinomial Sampling

In [7]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[0 0 0 ... 1 0 0]


In [8]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(graph)[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[89.04638  ,  4.7321377, 89.30968  ],
       [88.121735 ,  4.7321377, 90.7335   ],
       [87.05333  ,  4.7321377, 92.05287  ],
       [87.604805 ,  4.7321377, 91.40717  ],
       [83.627205 ,  5.2797017, 94.484726 ]], dtype=float32)

# KNN extended graph

In [9]:
"""
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)
"""

"\n_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points\n\nextended_graph = nx.Graph()\nfor index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node\n    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates\n    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)\n        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples\n        extended_graph.add_edge(*edge)                                      # Add the edge to the graph\n\ntransformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)\n"

In [10]:
def knn_and_extended_graph(list_k_nodes_i, k_neighbors=15):
    """
    Perform k-NN on the given points and construct an extended graph.

    Parameters:
    - list_k_nodes_i: array-like
        List of coordinates of the selected points.
    - k_neighbors: int, optional (default=15)
        Number of neighbors for k-NN.

    Returns:
    - extended_graph_i: networkx.Graph
        Extended graph.
    """

    # Perform k-NN on the selected points
    _, indices = NearestNeighbors(n_neighbors=k_neighbors).fit(list_k_nodes_i).kneighbors(list_k_nodes_i)

    # Construct the extended graph
    extended_graph_i = nx.Graph()
    for poly in indices:
        current_node = tuple(list_k_nodes_i[poly[0]])
        for index_other_node in poly[1:]:
            edge = current_node, tuple(list_k_nodes_i[index_other_node])
            extended_graph_i.add_edge(*edge)

    return extended_graph_i

extended_graph = knn_and_extended_graph(list_k_nodes)
transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 1500
Number of edges: 11807


# Edge Predictor

In [11]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(1500, 64)

In [12]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [13]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

In [14]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [15]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 8)	1
  (1, 9)	1
  (1, 13)	1
  (1, 14)	1
  :	:
  (1498, 1446)	1
  (1498, 1467)	1
  (1498, 1468)	1
  (1498, 1469)	1
  (1498, 1470)	1
  (1498, 1471)	1
  (1498, 1495)	1
  (1498, 1496)	1
  (1498, 1497)	1
  (1498, 1499)	1
  (1499, 1411)	1
  (1499, 1433)	1
  (1499, 1441)	1
  (1499, 1446)	1
  (1499, 1447)	1
  (1499, 1448)	1
  (1499, 1468)	1
  (1499, 1469)	1
  (1499, 1470)	1
  (1499, 1471)	1
  (1499, 1472)	1
  (1499, 1494)	1
  (1499, 1495)	1
  (1499, 1497)	1
  (1499, 1498)	1
[[0.07074653 0.07074291 0.07057048 ... 0.06997408 0.06999629 0.06997408]
 [1.14298693 0.07093672 0.07081278 ... 0.07038329 0.07039931 0.07038329]
 [1.10145439 0.54700737 0.06735767 ... 0.06773871 0.06772602 0.06773871]
 ...
 [1.2342367  1.22916099 1.36592742 ... 0.06751291 0.06751882 0.

In [16]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

In [17]:
print(A_s)  # symmétrique
print(A_s.shape)

[[4.63017665e-30 4.70811019e-30 4.53179123e-30 ... 8.20846974e-29
  8.45910320e-29 2.59123091e-28]
 [4.70811019e-30 4.78694793e-30 4.62836894e-30 ... 8.35398433e-29
  8.60631094e-29 2.65423983e-28]
 [4.53179123e-30 4.62836894e-30 4.47581103e-30 ... 8.04840968e-29
  8.29773180e-29 2.58304900e-28]
 ...
 [8.20846974e-29 8.35398433e-29 8.04840968e-29 ... 3.13745226e-27
  3.00809060e-27 5.91266368e-27]
 [8.45910320e-29 8.60631094e-29 8.29773180e-29 ... 3.00809060e-27
  2.91740316e-27 6.61199754e-27]
 [2.59123091e-28 2.65423983e-28 2.58304900e-28 ... 5.91266368e-27
  6.61199754e-27 6.68894608e-26]]
(1500, 1500)


# Face Classifier

### TriConv

#### Inputs

In [18]:
igraph_g = igraphGraph(directed=extended_graph.is_directed()).from_networkx(extended_graph)
print(igraph_g.summary())

IGRAPH U--- 1500 11807 -- 
+ attr: _nx_name (v)


In [19]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[903 905 910]
 [706 905 910]
 [705 706 910]]
32524


In [20]:
triangles = np.array(igraph_g.vs['_nx_name'])[triangles_ids_igraph]   #les triangles
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[78.294556  -7.328478  83.33951  ]
  [77.74096   -6.9390492 83.19142  ]
  [76.699425  -6.0434117 82.89942  ]]

 [[76.39445   -5.0095077 81.91051  ]
  [77.74096   -6.9390492 83.19142  ]
  [76.699425  -6.0434117 82.89942  ]]

 [[75.63655   -5.0095077 82.62972  ]
  [76.39445   -5.0095077 81.91051  ]
  [76.699425  -6.0434117 82.89942  ]]]
32524


In [21]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(triangles))
p_init = (A_s_ij + A_s_ik + A_s_jk) / 3

print(p_init.shape)
print(p_init)

(32524,)
[1.45071848e-26 1.65933740e-24 1.80662278e-24 ... 3.45038925e-22
 1.24997360e-18 3.44346422e-22]


#### Calculate barycenter

In [22]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(32524, 3)
[[77.578316  -6.770313  83.143456 ]
 [76.94495   -5.9973226 82.667114 ]
 [76.24348   -5.3541427 82.47988  ]]


#### KNN Tri

In [23]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [24]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(32524, 19, 3)


#### calculate e norm matrix

In [25]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(32524, 3)
[[0.6928588 2.0951166 1.4043591]
 [2.678986  1.4628468 1.4043591]]


#### Calculate r

In [26]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(32524,)

In [27]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(32524, 19)
(32524, 19)


In [28]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(32524, 19, 5)


#### Calculate f

In [29]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

0.9999999999998828
(32524,)
[3.07406003e-05 3.07406003e-05 3.07406003e-05 ... 3.07406003e-05
 3.07406003e-05 3.07406003e-05]


C:\Users\ahdel\AppData\Local\Temp\ipykernel_1496\474768578.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles


# Simplified Mesh

In [30]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[0])

(500, 3, 3)
[[39.        -7.9077644 93.86099  ]
 [51.         7.0402236 84.7203   ]
 [63.        -8.57404   88.01863  ]]


In [31]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 45
Number of edges: 224


In [32]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

#Affichage
transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)

# Fonctions Loss

## Probabilistic Chamfer distance

Récupération des points de l'espace de départ P

In [33]:
keys_input = np.array(graph)
print("Les points avant K-NN : ", keys_input)
print("dim points avant K-NN : ", len(keys_input))  #return 5999

Les points avant K-NN :  [[75.0252     -0.66553295  3.8       ]
 [75.01418    -0.49935842 16.7       ]
 [75.          0.          3.8       ]
 ...
 [51.         -8.57404    -6.9813724 ]
 [39.         -8.699333   -6.327254  ]
 [51.         -8.699333   -6.327254  ]]
dim points avant K-NN :  5999


Récupération des points de l'espace de départ Ps

In [34]:
keys_output = np.array(extended_graph)
print("sampled points : ", keys_output)
print("dim sample points : ", len(keys_output))  #return: 50

sampled points :  [[89.04638    4.7321377 89.30968  ]
 [89.33791    4.1574683 89.478    ]
 [88.71803    5.2797017 89.12011  ]
 ...
 [86.70247   -7.8332825 86.41028  ]
 [86.41375   -7.8332825 87.015594 ]
 [86.09374   -7.8332825 87.60497  ]]
dim sample points :  1500


Distance Matrix

In [35]:
def matrix_distance_euclidienne_barycentres(barycentre_b, barycentre_b_hat):
    """
    Calculate distances between barycenters b et b_hat after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """
    return cdist(barycentre_b, barycentre_b_hat, metric='sqeuclidean').T

distances_matrix = matrix_distance_euclidienne_barycentres(keys_input, keys_output)
print("Squared Euclidean Distances Matrix Shape:", distances_matrix.shape)
print("Distances Matrix:\n", distances_matrix)




Squared Euclidean Distances Matrix Shape: (1500, 5999)
Distances Matrix:
 [[ 7537.63320434  5496.4361785   7531.59878255 ... 10896.5475473
  11831.46702982 10774.35393168]
 [ 7568.83426078  5523.49200076  7563.57945505 ... 10936.29683024
  11877.84893302 10813.7390087 ]
 [ 7502.36059237  5465.86519297  7495.5807782  ... 10850.07099805
  11777.49538147 10728.26259339]
 ...
 [ 7012.19333615  5049.92526476  7022.76618401 ...  9997.21542201
  10876.52544754  9875.66619461]
 [ 7105.91087289  5128.01928199  7116.46916936 ... 10090.11216468
  10961.7010329   9967.77104023]
 [ 7197.16274648  5204.05814207  7207.70491465 ... 10178.6961734
  11041.83381922 10055.58400233]]


In [36]:
# Calculer le min des x de distances_matrix
def min_pour_x_list(distances_matrix, x_in):
    return np.min(distances_matrix[:, x_in])

# Calculer le min des y de distances_matrix
def min_pour_y_list(distances_matrix, y_out):
    return np.min(distances_matrix[y_out, :])


In [37]:
def d_P_Ps(keys_x, keys_y, p_y):
    """
    Calculate the Probabilistic Chamfer distance between two sets of points.

    Parameters:
    - keys_x (list): The input vertex set, a list of tuples.
    - keys_y (list): The sampled points, a list of tuples.
    - p_y (list): Their respective probabilities, a weight associated with each point in keys_y.

    Returns:
        chamfer_distance (float): The Probabilistic Chamfer distance between the two sets of points.
    """
    min_for_x = [p_y[i] * min_pour_x_list(distances_matrix, i) for i in range(len(keys_x))]
    min_for_y = [p_y[j] * min_pour_y_list(distances_matrix, j) for j in range(len(keys_y))]
    
    sum_for_y = np.sum(min_for_y)
    sum_for_x = np.sum(min_for_x)

    chamfer_distance = sum_for_y + sum_for_x
    
    return chamfer_distance


In [38]:
print("PROBA : ", normalized_inclusion_score)

PROBA :  [0.00021659 0.00021663 0.00021659 ... 0.00021254 0.00023529 0.00021254]


In [39]:
prob_chamfer_dist = d_P_Ps(keys_input, keys_output, normalized_inclusion_score)
print("Chamfer Distance:", prob_chamfer_dist)

Chamfer Distance: 5.014634247657209


## Probabilistic Surfaces Distance

barycentres b et b_hat

In [40]:
# Creation igraph
igraph_g_original = igraphGraph(directed=extended_graph.is_directed()).from_networkx(graph)
print(igraph_g_original.summary())

#Find triangles
triangles_ids_igraph_original = np.array(igraph_g_original.cliques(min=3, max=3))
triangles = np.array(igraph_g_original.vs['_nx_name'])[triangles_ids_igraph_original]
print("triangles shape : ", triangles.shape)
print(triangles[0])

# Calculate barycenters
b = np.mean(triangles, axis=1)
print("shape b : ", b.shape)
print("Barycentre b: ", b)

IGRAPH U--- 5999 17991 -- 
+ attr: _nx_name (v), index_triangle (v)
triangles shape :  (11994, 3, 3)
[[39.        8.793697 -4.666995]
 [51.        8.793697 -4.666995]
 [39.        8.793697 -5.333005]]
shape b :  (11994, 3)
Barycentre b:  [[43.          8.793697   -4.8889985 ]
 [55.          1.0515273   3.7313068 ]
 [55.         -8.515905   -2.804371  ]
 ...
 [75.05613    -0.94137865 21.        ]
 [75.01313    -0.3882971   8.099999  ]
 [75.03516    -0.72036856 25.300001  ]]


In [41]:
# Get finals selected triangles & calculate barycenters
b_hat = np.mean(selected_triangles, axis=1)
print(b_hat.shape)
print("B_HAT : ", b_hat)

(500, 3)
B_HAT :  [[ 51.         -3.147194   88.86664  ]
 [ 51.         -2.3899596  81.80843  ]
 [ 51.         -2.7398205  82.08131  ]
 ...
 [ 47.          4.1359315 -10.814036 ]
 [ 39.         -4.434086   -7.9858613]
 [ 43.         -3.6821384  -6.899863 ]]


test matrice des normes euclidiennes entre b et b_hat

In [42]:
distances = matrix_distance_euclidienne_barycentres(b, b_hat)
print(distances.shape)
print("Distances euclidiennes entre les barycentres b et b_hat :\n", distances)


(500, 11994)
Distances euclidiennes entre les barycentres b et b_hat :
 [[8996.70429575 7281.65391074 8448.39700934 ... 5189.44356367
  7107.49186888 4624.29556169]
 [7705.51890488 6123.88155183 7212.85392298 ... 4278.46133977
  6013.57033224 3773.67920109]
 [7760.85597958 6169.0967654  7254.94141432 ... 4312.85775595
  6055.39387341 3805.88355506]
 ...
 [  72.80085651  285.08055712  288.22371367 ... 1825.05838844
  1162.94485751 2113.77734174]
 [ 200.56480763  423.38397928  299.50909123 ... 2152.42363024
  1572.06886783 2420.27282077]
 [ 159.69005398  279.42935717  184.13835464 ... 1813.50954227
  1260.685761   2071.85455264]]


In [43]:
minimums_b = np.min(distances, axis=1)
print(minimums_b.shape)
print(minimums_b[:5])

(500,)
[45.69712659 16.11604068 16.17244063 16.5260575  16.96616842]


In [44]:
d_f_S_Ss = np.sum(selected_triangles_indexes * minimums_b)
print(d_f_S_Ss)

147748927.03996018


matrice sqeuclidean b

In [45]:
def normes_euclidiennes_carrees_b(bar_b):
    # Calculer la matrice des normes euclidiennes au carré
    normes_carrees = np.sum(bar_b**2, axis=1)

    return normes_carrees

matrix_b = normes_euclidiennes_carrees_b(b)
print(matrix_b.shape)
print(matrix_b)

(11994,)
[1950.2314 3040.0283 3105.3853 ... 6075.3086 5692.7305 6270.884 ]


In [49]:
def matrix_distances_after_knn(b_hat_n, number_neigh=3):
    """
    Calculate distances between barycenters after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """

    # Connect the extended graph using the k-NN process
    extended_graph = knn_and_extended_graph(b_hat_n, number_neigh)

    # Extract barycenters after the k-NN process
    b_hat_extended = np.array(list(extended_graph._node.keys()))

    # Calculate distances after k-NN
    distances_after = cdist(b_hat_extended, b_hat_extended)

    return distances_after

# Example usage with b_hat
distances_after = matrix_distances_after_knn(b_hat)

# Print the results
print("Distances After k-NN:\n", distances_after)

Distances After k-NN:
 [[ 0.          4.0750322   4.11605999 ... 94.85239976 93.48194688
  92.41829745]
 [ 4.0750322   0.          0.22200313 ... 95.12330824 93.75941502
  92.67036796]
 [ 4.11605999  0.22200313  0.         ... 95.34266996 93.97749569
  92.89024164]
 ...
 [94.85239976 95.12330824 95.34266996 ...  0.          3.86468243
   3.1547582 ]
 [93.48194688 93.75941502 93.97749569 ...  3.86468243  0.
   5.6318603 ]
 [92.41829745 92.67036796 92.89024164 ...  3.1547582   5.6318603
   0.        ]]


## Triangle Collision Loss

In [48]:
def L_c(p_t, m_c_t, Fs):
    """
    Compute the collision loss term L_c.

    Parameters:
    - p_t: 1D numpy array containing the probabilities of each triangle (indices)
    - m_c_t: 2D numpy array containing the number of faces penetrated by each triangle
    - Fs: 3D numpy array representing the vertices of triangles

    Returns:
    - L_c: Collision loss term
    """
    assert len(p_t) == len(m_c_t), "Input arrays must have the same length"

    penalty_per_triangle = p_t * m_c_t

    # Sum the penalties for all selected triangles
    total_penalty = np.sum(penalty_per_triangle)

    # Compute the collision loss term L_c
    L_c = (1 / len(Fs)) * total_penalty

    return L_c

# Example usage:
# Replace the arrays below with your actual data
# p_t = selected_triangles_indexes
# m_c_t = numpy array containing the number of faces penetrated by each triangle
# Fs = 3D numpy array representing the vertices of triangles
p_t = selected_triangles_indexes
m_c_t = np.random.rand(500, 1)  # Given data
Fs = triangles  # Given data

collision_loss = L_c(p_t, m_c_t, Fs)
print("Collision Loss:", collision_loss)


Collision Loss: 196152.67066575598
